In [2]:
!pip install pyspark

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6a19f13b06247773de47df75b7d542cb665be8bfb40b3d62d36c8da6267dcebe
  Stored in directory: c:\users\nunop\appdata\local\pip\cache\wheels\9f\f5\07\7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark

You should consider upgrading via the 'c:\users\nunop\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [3]:
import pyspark